# **Whisper Model fine-tuning for ASR**
fine-tuning of Wav2vec2 for automatic speech recognition task. In this script, we try to collect all the processes, which consist of preparing data, making LM, fine-tuning the Wav2vec2, and evaluating with and without LM. 

*** For making the LM, you need a corpus.txt file. This file could be a big text file such as a corpus of the Gigaspeech dataset, [TEDLIUM](https://www.openslr.org/27/), a mix of all of them, or a file consisting of all the sentences of the dataset that you are working on in this script.

**** To prepare the train.csv and test.csv files, you can find the manifest style in our other repository for [Whisper_fine_tuning_ASR](https://github.com/areffarhadi/Whisper_fine_tuning_ASR). The manifests of these two scripts are the same.

## **Preparing Data to fine-tune**

In [45]:
import numpy as np
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [ ]:
!pip install datasets
!pip install pandas
!pip install numpy
!pip install hazm
!pip install num2fawords
!pip install tqdm
!pip install scikit-learn
!pip install jiwer
!pip install transformers==4.28.0
!pip install urllib3==1.26.0
!pip install torchaudio
!pip install 
!pip install langchain
!pip install sentence-transformers==2.2.2

In [46]:
from datasets import load_dataset, load_metric

import pandas as pd
import numpy as np

#import hazm
from num2fawords import words, ordinal_words
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import os
import string
import six
import re
import glob

In [47]:
common_voice_train = load_dataset("csv", data_files={"train": "train.csv"}, delimiter=",")["train"]
common_voice_test = load_dataset("csv", data_files={"test": "test.csv"}, delimiter=",")["test"]
print(common_voice_train)
print(common_voice_test)

Dataset({
    features: ['Path', 'Sentence'],
    num_rows: 14300
})
Dataset({
    features: ['Path', 'Sentence'],
    num_rows: 718
})


Extract language characters based on the Sentences of csv files. 
After that, make the tokenizer, feature extractor, and processor.

In [48]:
def extract_all_chars(batch):
    all_text = " ".join(batch["Sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [49]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/14300 [00:00<?, ? examples/s]

Map:   0%|          | 0/718 [00:00<?, ? examples/s]

In [50]:
vocab_list = list(sorted(set(vocab_train["vocab"][0]) ))
vocab_list = [vocab for vocab in vocab_list if vocab not in [" ", "\u0307","<",">"]]
print(len(vocab_list))
print(vocab_list)

28
["'", '-', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [51]:
special_vocab = ["<pad>", "<s>", "</s>", "<unk>", "|"]

In [52]:
vocab_dict = {v: k for k, v in enumerate(special_vocab + vocab_list)}
print(len(vocab_dict))
print(vocab_dict)

33
{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, "'": 5, '-': 6, 'a': 7, 'b': 8, 'c': 9, 'd': 10, 'e': 11, 'f': 12, 'g': 13, 'h': 14, 'i': 15, 'j': 16, 'k': 17, 'l': 18, 'm': 19, 'n': 20, 'o': 21, 'p': 22, 'q': 23, 'r': 24, 's': 25, 't': 26, 'u': 27, 'v': 28, 'w': 29, 'x': 30, 'y': 31, 'z': 32}


In [53]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In this part, the model path and the save path of the final results are determined. 
Here, you can download the pretrained model [facebook/wav2vec2-large-xlsr-53](https://huggingface.co/facebook/wav2vec2-large-xlsr-53). However, you can also download another version of the wav2vec2 model. In addition, you can continue the training process based on your previous checkpoint.

In [54]:
model_name_or_path = "/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/wav2vec2-large-xlsr-53"
#model_name_or_path = "/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir/checkpoint-55000"

save_dir = "/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir"

In [55]:
from transformers.trainer_utils import get_last_checkpoint

last_checkpoint = None
if os.path.exists(save_dir):
    last_checkpoint = get_last_checkpoint(save_dir)

print(last_checkpoint if last_checkpoint else str(None))

/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir/checkpoint-55000


In [56]:
from transformers import Wav2Vec2CTCTokenizer

#if not os.path.exists(save_dir) and not model_name_or_path:
   # print("Load from scratch")
tokenizer = Wav2Vec2CTCTokenizer(
        "vocab.json",
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        word_delimiter_token="|",
        do_lower_case=False)
#else:
  #  print(f"Load from {model_name_or_path}")
    #tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("m3hrdadfi/wav2vec2-large-xlsr-persian-v3")


In [57]:
text = "do you hear the sleigh bells ringing"
print(" ".join(tokenizer.tokenize(text)))
print(tokenizer.decode(tokenizer.encode(text)))

d o | y o u | h e a r | t h e | s l e i g h | b e l l s | r i n g i n g
do you hear the sleigh bels ringing


In [58]:
from transformers import Wav2Vec2FeatureExtractor

if not os.path.exists(save_dir) and not model_name_or_path:
    print("Load from scratch")
    feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
else:
    print(f"Load from {model_name_or_path}")
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)

Load from /home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/wav2vec2-large-xlsr-53


In [59]:

from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [60]:
if len(processor.tokenizer.get_vocab()) == len(processor.tokenizer):
    print(len(processor.tokenizer))

33


## Resample your data!
The sampling rate at which the audio files should be digitalized is 16000 hertz (Hz).
Since the pre-trained [Wav2Vec2](https://huggingface.co/facebook/wav2vec2-large-xlsr-53)  model is trained with 16000 Hz sample rate audio files, we must resample our data accordingly. If your data is in 16000 Hz in this cell. 

In [61]:
import torchaudio
#import librosa


target_sampling_rate = 16000

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["Path"])
    speech_array = speech_array.squeeze().numpy()
    #speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, target_sampling_rate)


    batch["speech"] = speech_array
    batch["sampling_rate"] = target_sampling_rate
    batch["duration_in_seconds"] = len(batch["speech"]) / target_sampling_rate
    batch["target_text"] = batch["Sentence"]
    return batch

In [62]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names, num_proc=24)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names, num_proc=24)

In [63]:
print(common_voice_train[0]["sampling_rate"])
print(common_voice_test[0]["sampling_rate"])

16000
16000


In [65]:
import IPython.display as ipd
import numpy as np
import random

sample = common_voice_train
rand_int = random.randint(0, len(sample))

print("Target text:", sample[rand_int]["target_text"])
print("Input array shape:", np.asarray(sample[rand_int]["speech"]).shape)
print("Sampling rate:", sample[rand_int]["sampling_rate"])

ipd.Audio(data=np.asarray(sample[rand_int]["speech"]), autoplay=True, rate=16000)

Target text: a boring novel is a superb sleeping pill
Input array shape: (41497,)
Sampling rate: 16000


In [66]:

def prepare_dataset(batch):
    # Check that all files have the correct sampling rate
    assert (len(set(batch["sampling_rate"])) == 1), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
    
    # Extract input values with padding
    inputs = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0], padding=True, return_tensors="pt")
    batch["input_values"] = inputs.input_values
    
    # Process target text
    with processor.as_target_processor():
        labels = processor(batch["target_text"], padding=True, return_tensors="pt")
    batch["labels"] = labels.input_ids
    
    return batch

In [67]:
_common_voice_train = common_voice_train.map(
    prepare_dataset,
    remove_columns=common_voice_train.column_names,
    batch_size=8,
    num_proc=24,
    batched=True
)
_common_voice_test = common_voice_test.map(
    prepare_dataset,
    remove_columns=common_voice_test.column_names,
    batch_size=8,
    num_proc=24,
    batched=True
)

In [68]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [69]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Use a metric to evaluate your fine-tuning task, here we've chosen WER, also you can use CER!

In [70]:
wer_metric = load_metric("wer")

/home/rf/anaconda3/lib/python3.9/site-packages/datasets/load.py:752: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [71]:
import random


def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    if isinstance(label_str, list):
        if isinstance(pred_str, list) and len(pred_str) == len(label_str):
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str[index]}"')

        else:
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str}"')

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# **Set your model config**

In [72]:
from transformers import Wav2Vec2ForCTC


model = Wav2Vec2ForCTC.from_pretrained(
    model_name_or_path,
    attention_dropout=0.05,
    activation_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.01249,
    final_dropout=0.0,
    mask_time_prob=0.05,
    mask_time_length=10,
    mask_feature_prob=0,
    mask_feature_length=10,
    layerdrop=0.01377,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    ctc_zero_infinity=True,
    bos_token_id=processor.tokenizer.bos_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer.get_vocab())
)

Some weights of the model checkpoint at /home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_q.weight', 'quantizer.codevectors', 'project_q.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_

In [73]:
model.freeze_feature_extractor()

/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1643: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


Set the `TrainingArguments`specially` per_device_train_batch_size` & `per_device_eval_batch_size`to achieve best performance.

In [76]:
from transformers import TrainingArguments

# per_device_train_batch_size= 8,
training_args = TrainingArguments(
    output_dir=save_dir,
    group_by_length=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=True,
    save_steps=5000,
    eval_steps=1000,
    logging_steps=10,
    learning_rate=4e-4,
    warmup_steps=500,
    save_total_limit=2,
    dataloader_num_workers=24,
)
#fp16=True,
#dataloader_num_workers=24

# **Train**
Start training <br>
You have to use GPU!

In [77]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=_common_voice_train,
    eval_dataset=_common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [78]:
print("step1 - training...")
train_result = trainer.train()
print("step2 - ")

metrics = train_result.metrics
print("step3")
max_train_samples = len(_common_voice_train)
metrics["train_samples"] = min(max_train_samples, len(_common_voice_train))
print("step4 - saving the model...")
trainer.save_model()
print("model created!")
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()
tokenizer.save_pretrained(training_args.output_dir)

step1 - training...


/home/rf/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/rf/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You 

Step,Training Loss,Validation Loss,Wer
1000,2.962400,2.975894,1.000000
2000,1.624300,1.546467,0.944976
3000,0.875200,1.244335,0.799227
4000,0.353100,1.146602,0.687707
5000,0.389100,0.752382,0.585020
6000,0.204100,0.693471,0.497792
7000,0.221700,0.674388,0.480861
8000,0.686700,0.525434,0.430990
9000,0.210400,0.645967,0.442400
10000,0.259400,0.653631,0.410563


/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` o

reference: "women may never become completely equal to men"
predicted: ""
reference: "jeff's toy go-cart never worked"
predicted: ""
reference: "can the agency overthrow alien forces"
predicted: ""


...

/home/rf/anaconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is d

reference: "may i order a strawberry sundae after i eat dinner"
predicted: "may i order a strawberry sundae after i eat dinner"
reference: "to further his prestige he occasionally reads the wall street journal"
predicted: "to further his prestige he occasionally read the wall street journal"
reference: "who took the kayak down the bayou"
predicted: "who took the kayak down the bayou"


/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/home/rf/anaconda3/lib/python3.9/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` o

reference: "may i order a strawberry sundae after i eat dinner"
predicted: "may i order a strawberry sundae after i eat dinner"
reference: "most precincts had a third of the votes counted"
predicted: "most precincts had a thourd the votes counted"
reference: "nonprofit organizations have frequent fund raisers"
predicted: "lonnprofit organizations frequent fund raisers"
step2 - 
step3
step4 - saving the model...
model created!
***** train metrics *****
  epoch                    =          30.0
  total_flos               = 54521583259GF
  train_loss               =        0.1169
  train_runtime            =   10:22:35.08
  train_samples            =         14300
  train_samples_per_second =        11.484
  train_steps_per_second   =         2.871


('/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir/tokenizer_config.json',
 '/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir/special_tokens_map.json',
 '/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir/vocab.json',
 '/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir/added_tokens.json')

find result files on your final path

# **Make Language Model using KenLM**
In this section, we used cleaned, normalized, and processed Farsi text data to prepare a language model. It is better to use large-size text data and diverse in terms of structure and classification, which means that the data includes poetry, news, daily conversation, etc.

We have written a comprehensive data-cleaning program that I will add to the documentation.
KenLM also has some configurations such as `prune`.

using these options make your LM lighter and faster, improves your performance

In [ ]:
!pip install pypi-kenlm
!pip install pyctcdecode

In [ ]:
# DO NOT RUN  # AREF
!git clone https://github.com/kpu/kenlm.git
%cd kenlm
!mkdir -p build
%cd build
!cmake .. && make
%cd ../..

In [ ]:
# raw_text.txt: a normalized text corpus (each line should contain a sentence)
!kenlm/build/bin/lmplz -o 3 \
    </home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/dataset/train_clean_5/corpus.txt \
    >/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/makingdata_for_wav2vec/lm_tgsmall2.arpa

# **wraping the model processor with LM**
This section wraps the model processor with LM that you can use later to transcript more accurately.

In [ ]:
from pyctcdecode import build_ctcdecoder
from transformers import AutoProcessor
from transformers import Wav2Vec2ProcessorWithLM
import kenlm

model_id = "/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir"

# LM must be in arpa format (for now)
lm_path = "/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/makingdata_for_wav2vec/lm_tgsmall2.arpa"


processor = AutoProcessor.from_pretrained(model_id)
vocab_dict = processor.tokenizer.get_vocab()
sorted_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

decoder = build_ctcdecoder(
    list(sorted_dict.keys()),
    lm_path,
)

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

processor_with_lm.save_pretrained("/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir/processor_with_lm_wtimit_chains")

In [ ]:
import torch
import kenlm
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2ProcessorWithLM
from pyctcdecode import build_ctcdecoder
import pyctcdecode
model_name_or_path = "/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir" #import model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #choose device type
print(model_name_or_path, device)


processor = Wav2Vec2ProcessorWithLM.from_pretrained("/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir/processor_with_lm_gigaspeech") #load pretrained processor with LM
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path) #load finetuned model


def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["Path"])
    speech_array = speech_array.squeeze().numpy()
    #speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(
        batch["speech"],
        sampling_rate=processor.feature_extractor.sampling_rate,
        return_tensors="pt",
        padding=True
    )

    input_values = features.input_values
    attention_mask = features.attention_mask

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits #when we are trying to load model with LM we have to use logits instead of argmax(logits)

    batch["predicted"] = processor.batch_decode(logits.numpy()).text
    return batch



In [ ]:
import torchaudio
import librosa
from datasets import load_dataset
import numpy as np

dataset = load_dataset("csv", data_files={"test":"/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/test_output4.csv"}, delimiter=",")["test"]

dataset = dataset.map(speech_file_to_array_fn)

In [ ]:
result = dataset.map(predict, batched=True, batch_size=4)

In [ ]:
from datasets import load_metric
wer = load_metric("wer")
cer = load_metric("cer")
print("WER: {:.2f}".format(100 * wer.compute(predictions=result["predicted"], references=result["Sentence"]))) #refrence
print("CER: {:.2f}".format(100 * cer.compute(predictions=result["predicted"], references=result["Sentence"])))

In [ ]:
for i in range(len(result)):
    reference, predicted =  result["Sentence"][i], result["predicted"][i]
    if reference.strip() == predicted.strip():
        continue
    print("reference:", reference)
    print("predicted:", predicted)
    print('---')


In [ ]:
from datasets import load_metric
wer = load_metric("wer")
print("WER: {:.2f}".format(100 * wer.compute(predictions=result["predicted"], references=result["Sentence"])))

## Without LM

In [ ]:
import torch
import kenlm
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2ProcessorWithLM
from pyctcdecode import build_ctcdecoder
import pyctcdecode
model_name_or_path = "/home/rf/aref/Code_my_edit-20240208T101521Z-001/Code_my_edit/save_dir"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model_name_or_path, device)

processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path).to(device)


def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["Path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(
        batch["speech"],
        sampling_rate=processor.feature_extractor.sampling_rate,
        return_tensors="pt",
        padding=True
    )

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    pred_ids = torch.argmax(logits, dim=-1)

    batch["predicted_N_LM"] = processor.batch_decode(pred_ids)
    return batch



In [ ]:
result = dataset.map(predict, batched=True, batch_size=4)

In [ ]:
from datasets import load_metric
wer = load_metric("wer")
cer = load_metric("cer")
print("WER: {:.2f}".format(100 * wer.compute(predictions=result["predicted_N_LM"],
                                             references=result["Sentence"])))
print("CER: {:.2f}".format(100 * cer.compute(predictions=result["predicted_N_LM"],
                                             references=result["Sentence"])))

In [ ]:
for i in range(len(result)):
    reference, predicted_N_LM =  result["Sentence"][i], result["predicted_N_LM"][i]
    if reference.strip() == predicted_N_LM.strip():
        continue
    print("reference:", reference)
    print("predicted:", predicted_N_LM)
    print('---')